### LoRA
학습 시에는 일부 파라미터만 튜닝하지만, 해당 파라미터를 원래 모델에 얹는 형식이기 때문에 추론 시에는 원래 사용되는 모델의 사용량만큼의 리소스를 사용하게 됨

**`r`**  
- 원래의 모델을 작은 rank로 분해할 때 사용하는 매개변수로, 행렬에서 독립인 행 또는 열의 수를 의미  
- 커지면 학습 시간과 학습 파라미터 수가 증가, 작아지면 연산량이 줄어들지만 성능이 낮아질 수 있음  
- 하지만 r 값이 커진다고 해서 항상 성능 향상을 보장하는 것은 아님  

**`lora_alpha`**  
- rank와 함께 사용되는 변수  
- 학습한 low rank의 파라미터를 가중치 몇을 곱해서 모델에 반영할 것인지 즉, fine-tuning한 파라미터들의 영향력을 얼마나 키울 것인지  

In [1]:
from peft import LoraConfig, TaskType

peft_config = LoraConfig(task_type=TaskType.SEQ_CLS,
                         inference_mode=False,
                         r=16,
                         lora_alpha=32)

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("klue/roberta-small", num_labels=1)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
from peft import get_peft_model

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 886,273 || all params: 68,977,922 || trainable%: 1.2849


In [4]:
from dataset import preprocess

train_data = preprocess(task="train", data_path='../../train_preprocess_v1.csv', model_name='klue/roberta-small', scale=True)
valid_data = preprocess(task="train", data_path='../../dev_preprocess_v1.csv', model_name='klue/roberta-small', scale=True)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
tokenization: 100%|██████████| 9324/9324 [00:03<00:00, 2886.47it/s]
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
tokenization: 100%|██████████| 550/550 [00:00<00:00, 2690.05it/s]


In [5]:
import wandb

sweep_config = {
    "method": "bayes",
    "metric": {
        "name": "pearson_corr",
        "goal": "maximize"
    },
    "early_terminate": {
        'type': 'hyperband',
        's': 3,
        'eta': 3,
        'max_iter': 100
    }
}

parameters_dict = {
    'max_epoch': {'values': [1, 3, 5]},
    'batch_size': {'values': [8, 16]},
    'learning_rate': {
        'distribution': 'uniform',
        'min': 1e-5,
        'max': 1e-3
    },
    'weight_decay': {
        'distribution': 'uniform',
        'min': 0.001,
        'max': 0.1
    },
    'scheduler_patience': {'values': [3, 5, 7]}
}

sweep_config['parameters'] = parameters_dict

sweep_id = wandb.sweep(sweep_config, project='lora-roberta-large-sweeps-test')

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


Create sweep with ID: 4dau88s5
Sweep URL: https://wandb.ai/oceann010315/lora-roberta-large-sweeps-test/sweeps/4dau88s5


In [6]:
import numpy as np
from scipy.stats import pearsonr
from transformers import TrainingArguments, Trainer

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    
    predictions = logits.squeeze()
    labels = np.array(labels)

    pearson_corr, _ = pearsonr(predictions, labels)

    return {"pearson_corr": pearson_corr}

def train(config=None):
    with wandb.init(config=config):
        config = wandb.config
        
        training_arguments = TrainingArguments(
            output_dir='./peft/',
            report_to='wandb',
            overwrite_output_dir=False,
            num_train_epochs=config.max_epoch,
            # learning_rate=config.learning_rate,
            # weight_decay=config.weight_decay,
            eval_strategy="epoch",
            per_device_train_batch_size=config.batch_size,
            per_device_eval_batch_size=config.batch_size,
            lr_scheduler_type='cosine', # 'linear', 'cosine', 'cosine_with_restarts', 'polynomial', 'constant', 'constant_with_warmup', 'inverse_sqrt'
        )

        trainer = Trainer(
            model=model,
            args=training_arguments,
            train_dataset=train_data,
            eval_dataset=valid_data,
            compute_metrics=compute_metrics,
        )
    
        trainer.train()

In [7]:
wandb.agent(sweep_id, train, count=20)

wandb: Agent Starting Run: 2cbl674i with config:
wandb: 	batch_size: 16
wandb: 	learning_rate: 0.0007272648374404899
wandb: 	max_epoch: 5
wandb: 	scheduler_patience: 3
wandb: 	weight_decay: 0.015552416343030569
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).


Epoch,Training Loss,Validation Loss,Pearson Corr
1,0.053000,0.032683,0.858285
2,0.024200,0.021902,0.864438
3,0.019800,0.020829,0.877517
4,0.016000,0.019688,0.879824
5,0.013000,0.020658,0.882652


eval/loss,█▂▂▁▂
eval/pearson_corr,▁▃▇▇█
eval/runtime,▁█▃▇▄
eval/samples_per_second,█▁▆▂▅
eval/steps_per_second,█▁▆▂▅
train/epoch,▁▁▂▃▄▅▅▆▇██
train/global_step,▁▁▂▃▄▅▅▆▇██
train/grad_norm,█▂▃▂▁
train/learning_rate,█▆▄▂▁
train/loss,█▃▂▂▁
eval/loss,0.02066


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: x9dotmjj with config:
wandb: 	batch_size: 16
wandb: 	learning_rate: 0.0004336686175324083
wandb: 	max_epoch: 1
wandb: 	scheduler_patience: 7
wandb: 	weight_decay: 0.011304639489898924
wandb: Currently logged in as: oceann0315 (oceann010315). Use `wandb login --relogin` to force relogin


...

eval/loss,▁
eval/pearson_corr,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▆██
train/global_step,▁▆██
train/grad_norm,▁█
train/learning_rate,█▁
train/loss,█▁
eval/loss,0.0201


wandb: Agent Starting Run: 6qtupb2n with config:
wandb: 	batch_size: 16
wandb: 	learning_rate: 0.0007835565249095074
wandb: 	max_epoch: 1
wandb: 	scheduler_patience: 3
wandb: 	weight_decay: 0.08725169781123476


Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).


Epoch,Training Loss,Validation Loss


wandb: Ctrl + C detected. Stopping sweep.


In [ ]:
from peft import AutoPeftModelForSequenceClassification
import os
import json
from torch.utils.data import DataLoader
import torch
from pandas import read_csv

model_path = './peft/checkpoint-10000'

with open(os.path.join(model_path, 'adapter_config.json')) as f:
        model_config = json.load(f)
    
model = AutoPeftModelForSequenceClassification.from_pretrained(model_path, num_labels=1)

test_data = preprocess(task="test", data_path='../../test_preprocess_v1.csv', model_name=model_config['base_model_name_or_path'])
test_loader = DataLoader(test_data, shuffle=False)

device = 'cuda' if torch.cuda.is_available else 'cpu'
model.to(device)

outputs = []
for batch in test_loader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        predictions = model(**batch)
        outputs.append(predictions.logits)

outputs = list(round(float(i), 1) for i in torch.cat(outputs))

sample_csv = read_csv('../../sample_submission.csv')
sample_csv['target'] = outputs

sample_csv.to_csv(os.path.join(model_path, 'output.csv'), index=False)